In [1]:
import pandas as pd

In [3]:
csv = pd.read_csv('../tasks.csv')

In [4]:
csv

,Task,Original budget,Spent budget,Starting date,Deadline date,Status,recommendation
0,1,20000,30000,2023-10-01,2023-10-31,red,"""The current budget is higher than the maximu..."
1,2,15000,20000,2023-10-05,2023-11-15,red,None
2,3,25000,10000,2023-10-15,2023-11-30,green,"""The current budget is within the allocated m..."
3,4,18000,20000,2023-10-10,2023-12-01,green,None
4,5,30000,60000,2023-10-20,2023-12-15,red,None


In [ ]:
# original_budget = None
# starting_date = None
# deadline_date = None

In [5]:
original_budget = csv.iloc[1,1]

15000

In [8]:
starting_date = csv.iloc[1,3]
starting_date

'2023-10-05'

In [10]:
deadline_date = csv.iloc[1,4]
deadline_date

'2023-11-15'